# Working with SQLAchemy

In [75]:
## import modules
import os
from datetime import datetime
from flask import Flask, render_template, url_for, request, redirect, flash

## import flaks wtf module
from flask_wtf import Form
from wtforms.fields import StringField
from flask.ext.wtf.html5 import URLField
from wtforms.validators import DataRequired, url

## import sqlachemy
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import desc

In [76]:
## initiate the flask app
app = Flask(__name__)

## flash requires secret_key to carry out sessions
## to get a secret_key
## import os
## os.urandom(24)
app.config['SECRET_KEY'] = b'3\nHO\x00\xdd\xae0B\xae\xa7{}\xa5\xed+ 6\x80\x87\xcaP\xc9\xe2'

## Setting up SQLite database connection
basedir = os.path.abspath(os.path.dirname('__file__'))
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'thermos.db')
db = SQLAlchemy(app)

C:\Program Files\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:800: UserWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.
  warnings.warn('SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.')


In [77]:
## manage.py - this helps in managing the database.
from flask.ext.script import Manager, prompt_bool

manager = Manager(app)

## Create Database
@manager.command
def initdb():
    ## SQLAchemy command to create db
    db.create_all()
    ## Adding default users
    db.session.add(User(username="kamparia", email="somideolaoye@gmail.com"))
    db.session.commit()
    print('Initialized the Database')
    
## Drop Database
@manager.command
def dropdb():
    if prompt_bool("Are you sure you want to loose all your data"):
        db.drop_all()
        print('Dropped the database')

## Run script
if __name__ == '__main__':
    manager.run()

usage: __main__.py [-?] {shell,initdb,dropdb,runserver} ...
__main__.py: error: invalid choice: 'C:\\Users\\osomide\\AppData\\Roaming\\jupyter\\runtime\\kernel-a4a54c18-ae32-4678-88b6-f34f48170058.json' (choose from 'shell', 'initdb', 'dropdb', 'runserver')


SystemExit: 2

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [78]:
## Class for creating database tables in SQLite
## Create Bookmark table
class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    date = db.Column(db.DateTime, default=datetime.utcnow)
    description = db.Column(db.String(300))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
 
    @staticmethod
    def newest(num):
        return Bookmark.query.order_by(desc(Bookmark.date)).limit(num)

    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

## Create User Table
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    bookmarks = db.relationship('Bookmark', backref='user', lazy='dynamic')

    def __repr__(self):
        return "<User '{}'>".format(self.username)
    
## SQLAlchemy creates a database based on DB 
## db.create_all()
dropdb()
initdb()

Are you sure you want to loose all your data [n]: y
Dropped the database
Initialized the Database


In [79]:
'''
## Insert record into the database table using SQLAlchemy
## Add User & Bookmark
u=User(username='kamparia', email='somideolaoye@gmail.com')
bk = Bookmark(url="http://facebook.com", date=datetime.utcnow(), description="Facebook Social Network")
db.session.add(u, bk)
db.session.commit()

## Simple queries using SQLAchemy
Bookmark.query.all() ## retrieve all row
Bookmark.query.get(1) ## get by primary key
##Bookmark.query.filter_by(username="kamparia").all() ## query using where clause 
'''

'\n## Insert record into the database table using SQLAlchemy\n## Add User & Bookmark\nu=User(username=\'kamparia\', email=\'somideolaoye@gmail.com\')\nbk = Bookmark(url="http://facebook.com", date=datetime.utcnow(), description="Facebook Social Network")\ndb.session.add(u, bk)\ndb.session.commit()\n\n## Simple queries using SQLAchemy\nBookmark.query.all() ## retrieve all row\nBookmark.query.get(1) ## get by primary key\n##Bookmark.query.filter_by(username="kamparia").all() ## query using where clause \n'

In [80]:
## Class for managing the Flask-WTForms 
class BookmarkForm(Form):
    url = URLField('The URL for your bookmark:', validators=[DataRequired(), url()])
    description = StringField('Add an optional description:', validators=[DataRequired()])

    def validate(self):
        if not self.url.data.startswith("http://") or\
            self.url.data.startswith("https://"):
            self.url.data = "http://" + self.url.data

        if not Form.validate(self):
            return False

        if not self.description.data:
            self.description.data = self.url.data

        return True

In [81]:
## Fake Login
def logged_in_user():
    return User.query.filter_by(username='kamparia').first()

In [ ]:
@app.route('/')
@app.route('/index')
@app.route('/home')
def index():
    return render_template('index.html', user=logged_in_user(), new_bookmarks=Bookmark.newest(5))

@app.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)

@app.route('/add', methods=['GET', 'POST'])
def add():
    ## Form validation before submit
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        ## Store form variable to database
        bm = Bookmark(user=logged_in_user(), url=url, description=description)
        db.session.add(bm)
        db.session.commit()
        flash("Stored '{}'".format(description))
        return redirect(url_for('index'))
    return render_template('add.html', form=form)


## Error Handling
@app.errorhandler(404) ## 404 error - page not found
def page_not_found(e):
    return render_template('404.html'), 404

@app.errorhandler(500) ## 500 error - server error
def server_error(e):
    return render_template('500.html'), 500

## initiate the app function
if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Dec/2016 16:44:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2016 16:44:41] "GET /add HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2016 16:44:54] "GET /user/kamparia HTTP/1.1" 200 -
